In [173]:
import pandas as pd
import geopandas as gpd
import pathlib
from pathlib import Path
from tqdm import tqdm

import gzip
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
import xml.etree.ElementTree as ET

# to read the excel
from openpyxl import load_workbook
from openpyxl import Workbook

import datetime


import os
import re

import numpy as np

from collections import OrderedDict

# set the working directory
BASE_DIR = Path.cwd()

### Sample information

In [174]:
#Sample information
sample_pct = 0.05 # sample value in percentage
scale_factor = 100/(100*sample_pct)

#### Set the path to the simulation run folder and get the date

In [175]:
# get the first and the last iteration name
def get_iteration_count(simulation_run):
    rootPath = BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS")
    list_iter=[]
    for name in [entry for entry in os.listdir(rootPath) if os.path.isdir(os.path.join(rootPath, entry))]:
        list_iter.append(int(name.rsplit("it.")[1]))
    return max((list_iter)),min((list_iter))
# setup the BEAM model output folder name here
simulation_run = "sf-tscore-test-sample-inflated-personal-vehicle-mode-choice-details-added-tracing__cdb"
last_iter,first_iter = get_iteration_count(simulation_run)

# get the path of the simulation run and also get its creation date
def get_foldercreation_inf(folder_name):
    fname = pathlib.Path(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)), (str(last_iter)+".linkstats.csv.gz")))
    assert fname.exists(), f'No such file: {fname}'  # check that the file exists
    ctime = datetime.datetime.fromtimestamp(fname.stat().st_ctime)
    return ctime.strftime("%Y-%m-%d")
# get its creation date
date_time = get_foldercreation_inf(simulation_run)

#### Perform analysis for Bay Area

In [176]:
# get VMT, VHT and ratio from the last iteration of the linkstats file
class linkstats_to_df(object):
    def __init__(self,df):
        self.dataframe = df
        self.dataframe = self.dataframe.reset_index(drop=True)
        self.dataframe["volume"] = self.dataframe["volume"].fillna(0)
        self.dataframe["length"] = self.dataframe["length"].fillna(0)
        self.dataframe["vmt"] = self.dataframe["length"]*0.00062137*self.dataframe["volume"]*scale_factor
        self.VMT_sum = self.dataframe["vmt"].sum()
        # add congested speed (mph)
        self.dataframe["congested_speed_mph"] = (self.dataframe["length"]/self.dataframe["traveltime"])*2.23694
        # add free-flow speed (mph)
        self.dataframe["freeflow_speed_mph"] = self.dataframe["freespeed"]*2.23694
        # add vehicle hours of delay (free flow speed - congested speed)* volume
        self.dataframe["VHD"] = ((self.dataframe["freeflow_speed_mph"]-self.dataframe["congested_speed_mph"])*(self.dataframe["volume"])*20)/(60*60)
        self.VHD_sum = self.dataframe["VHD"].sum()
        # calculate VHT
        self.dataframe["VHT"] = self.dataframe["volume"]*(self.dataframe["traveltime"]/(60*60))*scale_factor
        self.VHT_sum = self.dataframe["VHT"].sum()
        # calculate VMT VHT Ratio
        self.VMT_VHT_ratio = self.VMT_sum/self.VHT_sum

dfBayAreaBeam = linkstats_to_df(pd.read_csv((BASE_DIR.parent.joinpath("runs",simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".linkstats.csv.gz")))))
# print("LinkStatistics")
# print("VMT: {}, VHT: {}, ratio:{}".format(dfBayAreaBeam.VMT_sum,dfBayAreaBeam.VHT_sum,dfBayAreaBeam.VMT_VHT_ratio))

now read the link stat from the SF County

In [177]:
### now read the link stat from the SF County
gdSFCountyNetwork = gpd.read_file((BASE_DIR.parent.joinpath("runs", simulation_run,"Network","SFCounty_EPSG4326.geojson")),driver='GeoJSON')
gdfSFBoundary = gpd.read_file((BASE_DIR.parent.joinpath("runs", simulation_run,"Network","SFCounty_boundary_EPSG4326.geojson")),driver='GeoJSON')

df = gdSFCountyNetwork.loc[~gdSFCountyNetwork["Information"].isin(['San Mateo', 'Golden Gate Bridge', 'SFOak1', 'SFOak2'])]
# # convert it to dictinary "Length" & "Link"
network_links = dict(zip(df["ID"], df["LENGTH"]))

dfBayArea = pd.read_csv(BASE_DIR.parent.joinpath((BASE_DIR.parent.joinpath("runs",simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".linkstats.csv.gz")))), compression="gzip", low_memory=True)
dfBayArea.reset_index(drop=True,inplace=True)
dfBayArea["link"]=dfBayArea["link"].astype(str)
dfSFCounty=dfBayArea[dfBayArea["link"].isin(network_links.keys())]
dfSFCountyBeam = linkstats_to_df(dfSFCounty)
# print("LinkStatistics")
# print("VMT: {}, VHT: {}, ratio:{}".format(dfSFCountyBeam.VMT_sum,dfSFCountyBeam.VHT_sum,dfSFCountyBeam.VMT_VHT_ratio))

read the iter.plans.xml file to get % of trips by modes

In [178]:
class detailed_events(object):
    def __init__(self,folder_path):
        self.dfEvents = pd.read_csv(folder_path,low_memory=False)
        self.dfEvents["Travel_Time"] = self.dfEvents["arrivalTime"] - self.dfEvents["departureTime"]
        self.TT_PTBoardings = {}
        PersonEnterVeh = self.dfEvents.loc[(self.dfEvents["type"]=="PersonEntersVehicle") &
                                           (~self.dfEvents["vehicle"].str.contains("rideHailVehicle",na=False)) &
                                           (~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False))].copy()
        # create dictionary: vehicle:mode
        veh_mode_df = self.dfEvents.loc[(self.dfEvents["type"]=="PathTraversal") &(self.dfEvents["vehicle"].str.contains("SF|BA", na=False))]
        veh_mode = dict(zip(veh_mode_df.vehicle, veh_mode_df["mode"]))
        # map them to get number of boardings by PT mode
        PersonEnterVeh["mode_veh"] = PersonEnterVeh["vehicle"].map(veh_mode,na_action='ignore')
        # get counts
        self.veh_counts = PersonEnterVeh[~PersonEnterVeh["mode_veh"].isna()].mode_veh.value_counts()
        self.bus_boarding = self.veh_counts.bus
        self.TT_PTBoardings["bus_boarding"] = self.veh_counts.bus
        self.tram_boarding = self.veh_counts.tram
        self.TT_PTBoardings["tram_boarding"] = self.veh_counts.tram
        self.rail_boarding = self.veh_counts.subway
        self.TT_PTBoardings["rail_boarding"] = self.veh_counts.subway
        self.cable_boarding = self.veh_counts.cable_car
        self.TT_PTBoardings["cable_boarding"] = self.veh_counts.cable_car

    def get_ii_modeshare(self):
        dfEvents = self.dfEvents[~self.dfEvents["person"].str.contains("ix",na=False) & ~self.dfEvents["person"].str.contains("rideHailAgent",na=False) & ~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~self.dfEvents["person"].isna())]
        return dfEvents.loc[(dfEvents["type"]=="TripArrivalEvent"),"mode"].value_counts(normalize=True)

    def get_TT_modeshare(self):
        dfEvents = self.dfEvents[~self.dfEvents["person"].str.contains("rideHailAgent",na=False) & ~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~self.dfEvents["person"].isna())]
        return dfEvents.loc[(dfEvents["type"]=="TripArrivalEvent"),"mode"].value_counts(normalize=True)

    def get_ii_PTBoardings(self):
        dfEvents_ii = self.dfEvents.copy()
        PTBoardings = {}
        dfEvents_ii = dfEvents_ii.loc[~dfEvents_ii["person"].str.startswith("ix", na=False)]
        PersonEnterVeh = dfEvents_ii.loc[(dfEvents_ii["type"]=="PersonEntersVehicle") &
                                         (~dfEvents_ii["vehicle"].str.contains("rideHailVehicle",na=False)) &
                                         (~dfEvents_ii["person"].str.contains("TransitDriverAgent",na=False))]
        pt_ridreship = len(PersonEnterVeh[PersonEnterVeh["vehicle"].str.contains("SF|BA", na=False)])
        # create dictionary: vehicle:mode
        veh_mode_df = dfEvents_ii.loc[(dfEvents_ii["type"]=="PathTraversal") & (dfEvents_ii["vehicle"].str.contains("SF|BA", na=False))]
        veh_mode = dict(zip(veh_mode_df.vehicle, veh_mode_df["mode"]))
        # map them to get number of boardings by PT mode
        PersonEnterVeh["mode_veh"] = PersonEnterVeh["vehicle"].map(veh_mode,na_action='ignore')
        # get counts
        veh_counts = PersonEnterVeh[~PersonEnterVeh["mode_veh"].isna()].mode_veh.value_counts()
        PTBoardings["bus_boarding"] = veh_counts.bus
        PTBoardings["tram_boarding"] =veh_counts.tram
        PTBoardings["rail_boarding"] =veh_counts.subway
        PTBoardings["cable_boarding"] = veh_counts.cable_car
        return PTBoardings

    def get_agents_count(self):
        agent_counts = {}
        agent_counts["ix_agents"]  = self.dfEvents.loc[self.dfEvents["person"].str.startswith("ix", na=False),"person"].nunique()
        agent_counts["ii_agents"]  = self.dfEvents.loc[(~ self.dfEvents["person"].str.startswith("ix", na=False)) & (~self.dfEvents["person"].str.startswith("rideHailAgent",na=False)) & (~self.dfEvents["person"].str.startswith("TransitDriverAgent",na=False)),"person"].nunique()
        agent_counts["ridehail_agent"] = self.dfEvents.loc[self.dfEvents["person"].str.startswith("rideHailAgent", na=False),"person"].nunique()
        agent_counts["total_agent"] = sum(agent_counts.values())
        return agent_counts

    def get_TT_unlinked_trips(self):
        dfEvents = self.dfEvents[~self.dfEvents["person"].str.contains("rideHailAgent",na=False) & ~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~self.dfEvents["person"].isna())]
        return dfEvents.loc[(dfEvents["type"]=="TripArrivalEvent"),"mode"].value_counts()

    def get_ii_unlinked_trips(self):
        dfEvents = self.dfEvents[~self.dfEvents["person"].str.contains("ix",na=False) & ~self.dfEvents["person"].str.contains("rideHailAgent",na=False) & ~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~self.dfEvents["person"].isna())]
        return dfEvents.loc[(dfEvents["type"]=="TripArrivalEvent"),"mode"].value_counts()

    def get_TT_tourpurpose_stats(self):
        dfEvents = self.dfEvents.copy()
        dfTour = dfEvents.loc[(~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                               ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                               (~dfEvents["person"].isna())),:]\
                         .dropna(subset = ["vehicleOwnership"])
        return dfTour\
            .groupby(["tourPurpose","vehicleOwnership","mode"]).size().reset_index().rename(columns={0:"count"})\
            .groupby("tourPurpose")\
            .apply(lambda vehOwntype: dict(vehOwntype.groupby("vehicleOwnership")
            .apply(lambda mode_used: dict(zip(mode_used["mode"],mode_used["count"])))))

    def get_ii_tourpurpose_stats(self):
        dfEvents = self.dfEvents.copy()
        dfTour = dfEvents.loc[(~dfEvents["person"].str.contains("ix",na=False) &
                               ~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                               ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                               (~dfEvents["person"].isna())),:]\
                            .dropna(subset = ["vehicleOwnership"])
        dfTour=dfTour[dfTour["type"]=="TripArrivalEvent"]
        tour_purpose_dict = dfTour.groupby(["tourPurpose", "vehicleOwnership", "mode"]).size().reset_index().rename(columns={0: "count"})\
                                           .groupby("tourPurpose")\
                                           .apply(lambda vehOwntype: dict(vehOwntype.groupby("vehicleOwnership")
                                                                                                 .apply(lambda mode_used: dict(zip(mode_used["mode"], mode_used["count"]))))).copy()
        for key,value in tour_purpose_dict.iteritems():
            if isinstance(tour_purpose_dict.get(key),dict):
                for auto_ownership,modes in tour_purpose_dict.get(key).items():
                    mode_keys = ["car", "walk_transit", "drive_transit", "bike_transit", "ride_hail_transit", "walk", "bike", "ride_hail_pooled", "ride_hail", "hov2_teleportation", "hov2","hov3_teleportation", "hov3"]
                    if isinstance(modes,dict):
                        for mode,count in modes.items():
                            if mode in mode_keys:
                                mode_keys.remove(mode)
                        for mode in mode_keys:
                            tour_purpose_dict[key][auto_ownership][mode]=0

        return tour_purpose_dict

    def get_ii_transfers(self):
        dfEvents = self.dfEvents.copy()
        dfEvents = dfEvents[~dfEvents["person"].str.contains("ix",na=False) & ~dfEvents["person"].str.contains("rideHailAgent",na=False) & ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~dfEvents["person"].isna())].copy()
        # dfEvents = dfEvents.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:].copy()

        dfEvents["numberofTransfers"]=dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[-22:].str.split(")",expand=True)[0].str.split(",",expand=True)[1]
        dfEvents["trip_cost_utility"]=dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[-34:-25].str.split(",",expand=True)[1]
        dfEvents["walkTransitDistances"] = dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[1:-37].str.split(",",expand=True)[1].str.split(":",expand=True)[0]
        # df["noofTransfers"]=df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:]["tracingData"].str[-22:].str.split(")",expand=True)[0].str.split(",",expand=True)[1]
        # df["trip_cost_utility"]=df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:]["tracingData"].str[-34:-25].str.split(",",expand=True)[1]
        # df["walkTransitDistance"] = df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:]["tracingData"].str[1:-37].str.split(",",expand=True)[1].str.split(":",expand=True)[0]
        # df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:][["tracingData","walkTransitDistance","trip_cost_utility","noofTransfers"]]

        # dfEvents["walkTransitDistances"]= dfEvents["tracingData"].str.split(":",expand=True)[0].str.split(",",expand=True)[1].str.split(")",expand=True)[0]
        # dfEvents["trip_cost_utility"]= dfEvents["tracingData"].str.split(":",expand=True)[1].str.split(",",expand=True)[1].str.split(")",expand=True)[0]
        # dfEvents["numberofTransfers"]= dfEvents["tracingData"].str.split(":",expand=True)[2].str.split(",",expand=True)[1].str.split(")",expand=True)[0]

        dfEvents["walkTransitDistances"] = pd.to_numeric(dfEvents["walkTransitDistances"], errors='coerce').replace(np.nan,0,regex=True)
        dfEvents["trip_cost_utility"] = pd.to_numeric(dfEvents["trip_cost_utility"], errors='coerce').replace(np.nan,0,regex=True)
        dfEvents["numberofTransfers"] = pd.to_numeric(dfEvents["numberofTransfers"], errors='coerce').replace(np.nan,0,regex=True).round(0).astype(int)

        dData = dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]

        dData["walkTransitDistances_mil"] = dData["walkTransitDistances"]*0.000621371

        def categorized_distance(cell_value):
            if cell_value>=1:
                walkTransitDistances_cat=">=1mil"
            elif ( cell_value<1.0) & (cell_value>0.5):
                walkTransitDistances_cat=">=0.5mil & <1mil"
            elif (cell_value<0.50) & (cell_value>0.25):
                walkTransitDistances_cat=">=0.25mil & <0.5mil"
            elif (cell_value<0.250):
                walkTransitDistances_cat="<0.25mil"
            return walkTransitDistances_cat

        dData["walkTransitDistances_cat"] = dData["walkTransitDistances_mil"].apply(lambda x:categorized_distance(x))

        def categorized_transfer(cell_value):
            if cell_value==0:
                categorized_transfer_cat="0"
            elif ( cell_value<=1.0) & (cell_value>0.0):
                categorized_transfer_cat="1"
            elif (cell_value<=2) & (cell_value>1):
                categorized_transfer_cat="2"
            elif (cell_value>=3):
                categorized_transfer_cat="3+"
            return categorized_transfer_cat

        dData["numberofTransfers_cat"] = dData["numberofTransfers"].apply(lambda x:categorized_transfer(x))

        transfer_dict = (dData.groupby(["tourPurpose", "numberofTransfers_cat","walkTransitDistances_cat"]).size().reset_index().rename(columns={0: "count"}).groupby("tourPurpose").apply(lambda vehOwntype: dict(vehOwntype.groupby("numberofTransfers_cat").apply(lambda mode_used: dict(zip(mode_used["walkTransitDistances_cat"], mode_used["count"]))))).copy())

        for key,values in transfer_dict.iteritems():
            # print(key)
            cat = ["0","1","2","3+"]
            for transfer_cat,dist in transfer_dict.get(key).items():
                # print("\t",transfer_cat)
                cat.remove(transfer_cat)
            for c in cat:
                # print(c)
                transfer_dict[key][c]={"<0.25mil":0,">=0.25mil & <0.5mil":0,">=0.5mil & <1mil":0,">=1mil":0 }

        for key,value in transfer_dict.iteritems():
            if isinstance(transfer_dict.get(key),dict):
                for transfer,walk_distance in transfer_dict.get(key).items():
                    walk_distance_keys = ["<0.25mil", ">=0.25mil & <0.5mil",">=0.5mil & <1mil",">=1mil"]
                    if isinstance(walk_distance,dict):
                        for cat,count in walk_distance.items():
                            if cat in walk_distance_keys:
                                walk_distance_keys.remove(cat)
                        for cat in walk_distance_keys:
                            transfer_dict[key][transfer][cat]=0

        return transfer_dict

    def get_ii_transfers_firstmile_summary(self):
        dfEvents = self.dfEvents.copy()
        dfEvents = dfEvents[~dfEvents["person"].str.contains("ix",na=False) & ~dfEvents["person"].str.contains("rideHailAgent",na=False) & ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~dfEvents["person"].isna())].copy()
        # dfEvents = dfEvents.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:].copy()

        dfEvents["numberofTransfers"]=dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[-22:].str.split(")",expand=True)[0].str.split(",",expand=True)[1]
        dfEvents["trip_cost_utility"]=dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[-34:-25].str.split(",",expand=True)[1]
        dfEvents["walkTransitDistances"] = dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[1:-37].str.split(",",expand=True)[1].str.split(":",expand=True)[0]

        dfEvents["walkTransitDistances"] = pd.to_numeric(dfEvents["walkTransitDistances"], errors='coerce').replace(np.nan,0,regex=True)
        dfEvents["trip_cost_utility"] = pd.to_numeric(dfEvents["trip_cost_utility"], errors='coerce').replace(np.nan,0,regex=True)
        dfEvents["numberofTransfers"] = pd.to_numeric(dfEvents["numberofTransfers"], errors='coerce').replace(np.nan,0,regex=True).round(0).astype(int)

        dData = dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]
        dData["walkTransitDistances_mil"] = dData["walkTransitDistances"]*0.000621371

        def categorized_distance(cell_value):
            if cell_value>=1:
                walkTransitDistances_cat=">=1mil"
            elif ( cell_value<1.0) & (cell_value>0.5):
                walkTransitDistances_cat=">=0.5mil & <1mil"
            elif (cell_value<0.50) & (cell_value>0.25):
                walkTransitDistances_cat=">=0.25mil & <0.5mil"
            elif (cell_value<0.250):
                walkTransitDistances_cat="<0.25mil"
            return walkTransitDistances_cat

        dData["walkTransitDistances_cat"] = dData["walkTransitDistances_mil"].apply(lambda x:categorized_distance(x))

        def categorized_transfer(cell_value):
            if cell_value==0:
                categorized_transfer_cat="0"
            elif ( cell_value<=1.0) & (cell_value>0.0):
                categorized_transfer_cat="1"
            elif (cell_value<=2) & (cell_value>1):
                categorized_transfer_cat="2"
            elif (cell_value>=3):
                categorized_transfer_cat="3+"
            return categorized_transfer_cat

        dData["numberofTransfers_cat"] = dData["numberofTransfers"].apply(lambda x:categorized_transfer(x))
        transfer_dict = (dData.groupby(["walkTransitDistances_cat","numberofTransfers_cat"]).size().reset_index().rename(columns={0: "count"}).groupby("walkTransitDistances_cat").apply(lambda mode_used: dict(zip(mode_used["numberofTransfers_cat"], mode_used["count"])))).copy()
        for key,values in transfer_dict.iteritems():
            cat = ["0","1","2","3+"]
            if isinstance(values,dict):
                for k,v in transfer_dict.get(key).items():
                    if k in cat:
                        cat.remove(k)
                for c in cat:
                    transfer_dict[key][c]=0

        return transfer_dict

    def get_ii_auto_ratio_status(self):
        dfEvents = self.dfEvents.copy()
        dfTour = dfEvents.loc[(~dfEvents["person"].str.contains("ix",na=False) &
                               ~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                               ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                               (~dfEvents["person"].isna())),:]\
                          .dropna(subset = ["vehicleOwnership"])
        return dfTour["vehicleOwnership"].value_counts(normalize=True)

    def get_TT_auto_ratio_status(self):
        dfEvents = self.dfEvents.copy()
        dfTour = dfEvents.loc[(~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                               ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                               (~dfEvents["person"].isna())),:]\
                        .dropna(subset = ["vehicleOwnership"])
        return dfTour["vehicleOwnership"].value_counts(normalize=True)

    def get_TT_activity_type(self):
        dfEvents = self.dfEvents.copy()
        dfActivity = dfEvents.loc[(~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                               ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                               (~dfEvents["person"].isna())),:]\
                         .dropna(subset = ["tourPurpose"])
        return dfActivity["tourPurpose"].value_counts(normalize=True)

    def get_ii_activity_type(self):
        dfEvents = self.dfEvents.copy()
        dfActivity = dfEvents.loc[(~dfEvents["person"].str.contains("ix",na=False) &
                                   ~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                                   ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                                   (~dfEvents["person"].isna())),:]\
                             .dropna(subset = ["tourPurpose"])
        return dfActivity["tourPurpose"].value_counts(normalize=True)

dfBayAreaBeamEvents = detailed_events(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".events.csv.gz")))
dfBayAreaZerothEvents = detailed_events(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(first_iter)),(str(first_iter)+".events.csv.gz")))

In [179]:
dfLast = dfBayAreaBeamEvents.dfEvents
dfFirst = dfBayAreaZerothEvents.dfEvents

dfLast.loc[(~dfLast["person"].str.contains("ix",na=False)) & (~dfLast["person"].str.contains("rideHailAgent",na=False))
           & (~dfLast["person"].str.contains("TransitDriverAgent",na=False)) & (~dfLast["person"].isna())
           & (dfLast["type"]=="TripArrivalEvent"),"mode"].value_counts().sum()

dfFirst.loc[(~dfFirst["person"].str.contains("ix",na=False)) & (~dfFirst["person"].str.contains("rideHailAgent",na=False))
            & (~dfFirst["person"].str.contains("TransitDriverAgent",na=False)) & (~dfFirst["person"].isna())
            & (dfFirst["type"]=="TripArrivalEvent"),"mode"].value_counts().sum()

33290

In [180]:
# dData = dfBayAreaBeamEvents.get_ii_transfers_firstmile_summary()
# transfer_dict = (dData.groupby(["walkTransitDistances_cat","numberofTransfers_cat"]).size().reset_index().rename(columns={0: "count"}).groupby("walkTransitDistances_cat").apply(lambda mode_used: dict(zip(mode_used["numberofTransfers_cat"], mode_used["count"])))).copy()
# for key,values in transfer_dict.iteritems():
#     # print(key)
#     cat = ["0","1","2","3+"]
#     if isinstance(values,dict):
#         for k,v in transfer_dict.get(key).items():
#             print(k,"-->",v)
#             if k in cat:
#                 cat.remove(k)
#         for c in cat:
#             transfer_dict[key][c]=0
# # Populate noofTransfers by tourpurpose
# transfer_wrksheet = workbook["noTransfer_Summary"]
# for distance_cat,values in transfer_dict.iteritems():
#     if isinstance((values,dict)):
#         cell_row=11
#         cell_col="D"
#         for k,v in values.items():


In [181]:
# Lets export these values in csv format
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
# read the excel template
workbook = load_workbook(filename=BASE_DIR.parent.joinpath("template", "templatev10.0.xlsx"))
sheet = workbook["Summary"]

In [182]:
# Summary worksheet
# general information
iteration_counter = 1
cell_row=1
cell_column=1
sheet.cell(row=(cell_row),column=(cell_column+1)).value = ("BEAM Simulation iteration {}".format(iteration_counter))
# Folder
sheet.cell(row=(cell_row+1),column=(cell_column+1)).value = simulation_run
# Date
sheet.cell(row=(cell_row+2),column=(cell_column+1)).value ="{}".format(date_time)
# Notes
sheet.cell(row=(cell_row+3),column=(cell_column+1)).value = ""
#Sample information
# sample_pct = 0.05
sheet.cell(row=(cell_row+6),column=(cell_column+1)).value = sample_pct*100
sheet.cell(row=(cell_row+7),column=(cell_column+1)).value = 100/(100*sample_pct)
# scale_factor = 100/(100*sample_pct)

In [183]:
# Model Run - Last Iteration
# dump values for full Bay Area
#change the heading of the simulation run
#re-locate the fixed row and column = 11
cell_row=11
cell_column=6

temp = "Beam_Run_"+date_time
sheet.cell(row=(cell_row),column=(cell_column)).value = temp
# sheet.merge_cells('D2:E2')
# sheet["F2"].font = Font(b=True,size=10)

# VMT & VHT Data
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeam.VMT_sum
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeam.VHT_sum
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeam.VMT_VHT_ratio
# sheet["F9"] = (VHD_sum)

# Mode Share data
cell_row+=2
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["car_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare().get("car",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["transit_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamEvents.get_TT_modeshare().get("walk_transit",0) +
                                                         dfBayAreaBeamEvents.get_TT_modeshare().get("bike_transit",0) +
                                                         dfBayAreaBeamEvents.get_TT_modeshare().get("drive_transit",0) +
                                                         dfBayAreaBeamEvents.get_TT_modeshare().get("rideh_hail_transit",0))
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare()["walk_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare().get("walk",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare()["bike_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare().get("bike",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["ride_hail_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare().get("ride_hail",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare().get("ride_hail_pool",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare().get("hov2",0) + \
                                                       dfBayAreaBeamEvents.get_TT_modeshare().get("hov2_teleportation",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare().get("hov3",0) +\
                                                       dfBayAreaBeamEvents.get_TT_modeshare().get("hov3_teleportation",0)

# Boarding data
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=(dfBayAreaBeamEvents.TT_PTBoardings["bus_boarding"]*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.TT_PTBoardings["tram_boarding"]+dfBayAreaBeamEvents.TT_PTBoardings["cable_boarding"])*scale_factor
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.TT_PTBoardings["rail_boarding"]*scale_factor)

# Agent stats
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_agents_count()["ix_agents"]*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_agents_count()["ii_agents"]*scale_factor)
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_agents_count().get("ridehail_agent",0)*scale_factor)

# Unlinked trip information
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=  ((dfBayAreaBeamEvents.get_TT_unlinked_trips().get("car",0))*scale_factor)
# Transit
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_unlinked_trips().get("walk_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_unlinked_trips().get("drive_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_unlinked_trips().get("bike_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_unlinked_trips().get("ridehail_transit",0) *scale_factor)
# Pedestrian
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_unlinked_trips().get("walk",0)*scale_factor)
# Bike
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_unlinked_trips().get("bike",0)*scale_factor)
# Ridehail
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_unlinked_trips().get("ride_hail",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_unlinked_trips().get("ride_hail_pooled",0)*scale_factor)
# HOV2 + HOV2_teleport
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_unlinked_trips().get("hov2",0)+
                                                         dfBayAreaBeamEvents.get_TT_unlinked_trips().get("hov2_teleportation",0))*scale_factor
# HOV3 + HOV3_teleport
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_unlinked_trips().get("hov3",0)+
                                                         dfBayAreaBeamEvents.get_TT_unlinked_trips().get("hov3_teleportation",0))*scale_factor

# "Total Trip Information"
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value=  (sum(dfBayAreaBeamEvents.get_TT_unlinked_trips())*scale_factor)

# Auto ownership status
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_auto_ratio_status().get("no_auto",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_auto_ratio_status().get("auto_sufficient",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_auto_ratio_status().get("auto_deficient",0))

# Activity Type
cell_row+=2
# sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("home",0))
# cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("othmaint",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("escort",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("shopping",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("social",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("work",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("eatout",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("othdiscr",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("univ",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("school",0))

In [184]:
# SFChamp Run - First Iteration
# dump values for full Bay Area
#change the heading of the simulation run
#re-locate the fixed row and column = 11
cell_row=11
cell_column=9

temp = "SF_Champ_"+date_time
sheet.cell(row=(cell_row),column=(cell_column)).value = temp
# sheet.merge_cells('D2:E2')
# sheet["F2"].font = Font(b=True,size=10)

# Mode Share data
cell_row+=5
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["car_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_TT_modeshare().get("car",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["transit_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.get_TT_modeshare().get("walk_transit",0) +
                                                         dfBayAreaZerothEvents.get_TT_modeshare().get("bike_transit",0) +
                                                         dfBayAreaZerothEvents.get_TT_modeshare().get("drive_transit",0) +
                                                         dfBayAreaZerothEvents.get_TT_modeshare().get("rideh_hail_transit",0))
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare()["walk_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_TT_modeshare().get("walk",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare()["bike_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_TT_modeshare().get("bike",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["ride_hail_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_TT_modeshare().get("ride_hail",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_TT_modeshare().get("ride_hail_pool",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_TT_modeshare().get("hov2",0) +\
                                                       dfBayAreaZerothEvents.get_TT_modeshare().get("hov2_teleportation",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_TT_modeshare().get("hov3",0) +\
                                                       dfBayAreaZerothEvents.get_TT_modeshare().get("hov3_teleportation",0)

# Boarding data
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=(dfBayAreaZerothEvents.TT_PTBoardings["bus_boarding"]*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.TT_PTBoardings["tram_boarding"]+
                                                        dfBayAreaZerothEvents.TT_PTBoardings["cable_boarding"])*scale_factor
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.TT_PTBoardings["rail_boarding"]*scale_factor)

# Agent stats
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.get_agents_count()["ix_agents"]*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.get_agents_count()["ii_agents"]*scale_factor)
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.get_agents_count().get("ridehail_agent",0)*scale_factor)

# Unlinked trip information
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=  ((dfBayAreaZerothEvents.get_TT_unlinked_trips().get("car",0))*scale_factor)
# Transit
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_unlinked_trips().get("walk_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_unlinked_trips().get("drive_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_unlinked_trips().get("bike_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_unlinked_trips().get("ridehail_transit",0) *scale_factor)
# Pedestrian
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_unlinked_trips().get("walk",0)*scale_factor)
# Bike
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_unlinked_trips().get("bike",0)*scale_factor)
# Ridehail
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_unlinked_trips().get("ride_hail",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_unlinked_trips().get("ride_hail_pooled",0)*scale_factor)
# HOV2 + HOV2_teleport
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_unlinked_trips().get("hov2",0)+
                                                         dfBayAreaZerothEvents.get_TT_unlinked_trips().get("hov2_teleportation",0))*scale_factor
# HOV3 + HOV3_teleport
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_unlinked_trips().get("hov3",0)+
                                                         dfBayAreaZerothEvents.get_TT_unlinked_trips().get("hov3_teleportation",0))*scale_factor

# "Total Trip Information"
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value=  (sum(dfBayAreaZerothEvents.get_TT_unlinked_trips())*scale_factor)

# Auto ownership status
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_auto_ratio_status().get("no_auto",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_auto_ratio_status().get("auto_sufficient",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_auto_ratio_status().get("auto_deficient",0))

# Activity Type
cell_row+=2
# sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_activity_type().get("home",0))
# cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_activity_type().get("othmaint",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_activity_type().get("escort",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_activity_type().get("shopping",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_activity_type().get("social",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_activity_type().get("work",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_activity_type().get("eatout",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_activity_type().get("othdiscr",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_activity_type().get("univ",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_TT_activity_type().get("school",0))

In [185]:
# Model Run - Last Iteration
# dump values for SF County
#change the heading of the simulation run
#re-locate the fixed row and column = 11
cell_row=11
cell_column=16

temp = "Beam_Run_County_"+date_time
sheet.cell(row=(cell_row),column=(cell_column)).value = temp
# sheet.merge_cells('D2:E2')
# sheet["F2"].font = Font(b=True,size=10)

# VMT & VHT Data
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfSFCountyBeam.VMT_sum
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfSFCountyBeam.VHT_sum
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfSFCountyBeam.VMT_VHT_ratio
# sheet["F9"] = (VHD_sum)

# Mode Share data
cell_row+=2
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["car_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_ii_modeshare().get("car",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["transit_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamEvents.get_ii_modeshare().get("walk_transit",0) +
                                                         dfBayAreaBeamEvents.get_ii_modeshare().get("bike_transit",0) +
                                                         dfBayAreaBeamEvents.get_ii_modeshare().get("drive_transit",0) +
                                                         dfBayAreaBeamEvents.get_ii_modeshare().get("rideh_hail_transit",0))
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare()["walk_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_ii_modeshare().get("walk",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare()["bike_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_ii_modeshare().get("bike",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["ride_hail_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_ii_modeshare().get("ride_hail",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_ii_modeshare().get("ride_hail_pool",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_ii_modeshare().get("hov2",0) +\
                                                       dfBayAreaBeamEvents.get_ii_modeshare().get("hov2_teleportation",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_ii_modeshare().get("hov3",0) +\
                                                       dfBayAreaBeamEvents.get_ii_modeshare().get("hov3_teleportation",0)

# Boarding data
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=(dfBayAreaBeamEvents.get_ii_PTBoardings().get("bus_boarding",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_ii_PTBoardings().get("tram_boarding",0)+
                                                        dfBayAreaBeamEvents.get_ii_PTBoardings().get("cable_boarding",0))*scale_factor
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_ii_PTBoardings().get("rail_boarding",0)*scale_factor)

# Agent stats
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_agents_count()["ii_agents"]*scale_factor)
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_agents_count().get("ridehail_agent",0)*scale_factor)

# Unlinked trip information
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=  ((dfBayAreaBeamEvents.get_ii_unlinked_trips().get("car",0))*scale_factor)
# Transit
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_unlinked_trips().get("walk_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_unlinked_trips().get("drive_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_unlinked_trips().get("bike_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_unlinked_trips().get("ridehail_transit",0) *scale_factor)
# Pedestrian
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_unlinked_trips().get("walk",0)*scale_factor)
# Bike
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_unlinked_trips().get("bike",0)*scale_factor)
# Ridehail
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_unlinked_trips().get("ride_hail",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_unlinked_trips().get("ride_hail_pooled",0)*scale_factor)
# HOV2 + HOV2_teleport
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_unlinked_trips().get("hov2",0)+
                                                         dfBayAreaBeamEvents.get_ii_unlinked_trips().get("hov2_teleportation",0))*scale_factor
# HOV3 + HOV3_teleport
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_unlinked_trips().get("hov3",0)+
                                                         dfBayAreaBeamEvents.get_ii_unlinked_trips().get("hov3_teleportation",0))*scale_factor

# "Total Trip Information"
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value=  (sum(dfBayAreaBeamEvents.get_ii_unlinked_trips())*scale_factor)

# Auto ownership status
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_auto_ratio_status().get("no_auto",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_auto_ratio_status().get("auto_sufficient",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_auto_ratio_status().get("auto_deficient",0))

# Activity Type
cell_row+=2
# sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("home",0))
# cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_activity_type().get("othmaint",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_activity_type().get("escort",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_activity_type().get("shopping",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_activity_type().get("social",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_activity_type().get("work",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_activity_type().get("eatout",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_activity_type().get("othdiscr",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_activity_type().get("univ",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_ii_activity_type().get("school",0))

c:\users\goyal\.virtualenvs\t-score-0msqhlpo\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [186]:
# SF Champ - First Iteration
# dump values for SF County
#change the heading of the simulation run
#re-locate the fixed row and column = 11
cell_row=11
cell_column=19

temp = "SF_Champ_County_"+date_time
sheet.cell(row=(cell_row),column=(cell_column)).value = temp
# sheet.merge_cells('D2:E2')
# sheet["F2"].font = Font(b=True,size=10)

# VMT & VHT Data
# cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfSFCountyBeam.VMT_sum
# cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfSFCountyBeam.VHT_sum
# cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfSFCountyBeam.VMT_VHT_ratio
# sheet["F9"] = (VHD_sum)

# Mode Share data
cell_row+=5
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["car_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_ii_modeshare().get("car",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["transit_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.get_ii_modeshare().get("walk_transit",0) +
                                                         dfBayAreaZerothEvents.get_ii_modeshare().get("bike_transit",0) +
                                                         dfBayAreaZerothEvents.get_ii_modeshare().get("drive_transit",0) +
                                                         dfBayAreaZerothEvents.get_ii_modeshare().get("rideh_hail_transit",0))
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare()["walk_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_ii_modeshare().get("walk",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare()["bike_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_ii_modeshare().get("bike",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["ride_hail_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_ii_modeshare().get("ride_hail",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_ii_modeshare().get("ride_hail_pool",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_ii_modeshare().get("hov2",0) +\
                                                       dfBayAreaZerothEvents.get_ii_modeshare().get("hov2_teleportation",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_ii_modeshare().get("hov3",0) +\
                                                       dfBayAreaZerothEvents.get_ii_modeshare().get("hov3_teleportation",0)

# Boarding data
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=(dfBayAreaZerothEvents.get_ii_PTBoardings().get("bus_boarding",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.get_ii_PTBoardings().get("tram_boarding",0)+
                                                        dfBayAreaZerothEvents.get_ii_PTBoardings().get("cable_boarding",0))*scale_factor
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.get_ii_PTBoardings().get("rail_boarding",0)*scale_factor)

# Agent stats
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.get_agents_count()["ii_agents"]*scale_factor)
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.get_agents_count().get("ridehail_agent",0)*scale_factor)

# Unlinked trip information
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=  ((dfBayAreaZerothEvents.get_ii_unlinked_trips().get("car",0))*scale_factor)
# Transit
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_unlinked_trips().get("walk_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_unlinked_trips().get("drive_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_unlinked_trips().get("bike_transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_unlinked_trips().get("ridehail_transit",0) *scale_factor)
# Pedestrian
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_unlinked_trips().get("walk",0)*scale_factor)
# Bike
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_unlinked_trips().get("bike",0)*scale_factor)
# Ridehail
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_unlinked_trips().get("ride_hail",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_unlinked_trips().get("ride_hail_pooled",0)*scale_factor)
# HOV2 + HOV2_teleport
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_unlinked_trips().get("hov2",0)+
                                                         dfBayAreaZerothEvents.get_ii_unlinked_trips().get("hov2_teleportation",0))*scale_factor
# HOV3 + HOV3_teleport
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_unlinked_trips().get("hov3",0)+
                                                         dfBayAreaZerothEvents.get_ii_unlinked_trips().get("hov3_teleportation",0))*scale_factor

# "Total Trip Information"
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value=  (sum(dfBayAreaZerothEvents.get_ii_unlinked_trips())*scale_factor)

# Auto ownership status
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_auto_ratio_status().get("no_auto",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_auto_ratio_status().get("auto_sufficient",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_auto_ratio_status().get("auto_deficient",0))

# Activity Type
cell_row+=2
# sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamEvents.get_TT_activity_type().get("home",0))
# cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_activity_type().get("othmaint",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_activity_type().get("escort",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_activity_type().get("shopping",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_activity_type().get("social",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_activity_type().get("work",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_activity_type().get("eatout",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_activity_type().get("othdiscr",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_activity_type().get("univ",0))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaZerothEvents.get_ii_activity_type().get("school",0))

In [187]:
# Lets export these values in csv format
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment

# read the excel template
# workbook = load_workbook(filename=BASE_DIR.parent.joinpath("template", "templatev7.0.xlsx"))
Mode_Choice_wrksheet = workbook["Mode_Choice"]

def nested_dict_pairs_populator(dict_obj,row_no,col_no):
    # Mode_Choice_wrksheet = sorted(dict_obj.items(),key=lambda kv:kv[1])
    modechoice_summary = OrderedDict(sorted(dict_obj.items())).copy()
    cell_row=row_no
    cell_column=col_no
    for mode, count in modechoice_summary.items():
        if "bike" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "bike_transit" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "car" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "drive_transit" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "hov2" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "hov2_teleportation" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "hov3" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "hov3_teleportation" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "ride_hail" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "ride_hail_pooled" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "ride_hail_transit" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "walk" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        elif "walk_transit" in mode:
            Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
            cell_column+=1
        else:
            continue
modechoice_summary = dfBayAreaBeamEvents.get_ii_tourpurpose_stats()
for tour_purpose in modechoice_summary.keys():
    if "work" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =11
            if "no_auto" in auto_ownership:
                cell_row="D"
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "school" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =29
            if "no_auto" in auto_ownership:
                cell_row="D"
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "univ" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =47
            if "no_auto" in auto_ownership:
                cell_row="D"
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "shopping" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =65
            if "no_auto" in auto_ownership:
                cell_row="D"
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "othmaint" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =83
            if "no_auto" in auto_ownership:
                cell_row="D"
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "eatout" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =101
            if "no_auto" in auto_ownership:
                cell_row="D"
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "social" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =119
            if "no_auto" in auto_ownership:
                cell_row="D"
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "othdiscr" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =137
            if "no_auto" in auto_ownership:
                cell_row="D"
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "escort" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =155
            if "no_auto" in auto_ownership:
                cell_row="D"
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)

modechoice_summary = dfBayAreaZerothEvents.get_ii_tourpurpose_stats()
for tour_purpose in modechoice_summary.keys():
    if "work" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =11
            if "no_auto" in auto_ownership:
                cell_row="L"
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "school" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =29
            if "no_auto" in auto_ownership:
                cell_row="L"
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "univ" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =47
            if "no_auto" in auto_ownership:
                cell_row="L"
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "shopping" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =65
            if "no_auto" in auto_ownership:
                cell_row="L"
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "othmaint" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =83
            if "no_auto" in auto_ownership:
                cell_row="L"
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "eatout" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =101
            if "no_auto" in auto_ownership:
                cell_row="L"
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "social" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =119
            if "no_auto" in auto_ownership:
                cell_row="L"
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "othdiscr" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =137
            if "no_auto" in auto_ownership:
                cell_row="L"
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)
    elif "escort" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            cell_column =155
            if "no_auto" in auto_ownership:
                cell_row="L"
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
            nested_dict_pairs_populator(modechoice_summary[tour_purpose][auto_ownership],cell_row,cell_column)

In [188]:
# Populate noofTransfers by tourpurpose
transfer_wrksheet = workbook["noofTransfers_tourpurpose"]

def nested_dict_pairs_populator(dict_obj,row_no,col_no):
    # Mode_Choice_wrksheet = sorted(dict_obj.items(),key=lambda kv:kv[1])
    transfer_summary = OrderedDict(sorted(dict_obj.items())).copy()
    cell_row=row_no
    cell_column=col_no
    for mode, count in transfer_summary.items():
        if "<0.25mil" in mode:
            transfer_wrksheet["{}{}".format(cell_row,cell_column)].value=int(count)*scale_factor
            cell_column+=1
        elif ">=0.25mil & <0.5mil" in mode:
            transfer_wrksheet["{}{}".format(cell_row,cell_column)].value=int(count)*scale_factor
            cell_column+=1
        elif ">=0.5mil & <1mil" in mode:
            transfer_wrksheet["{}{}".format(cell_row,cell_column)].value=int(count)*scale_factor
            cell_column+=1
        elif ">=1mil" in mode:
            transfer_wrksheet["{}{}".format(cell_row,cell_column)].value=int(count)*scale_factor
            cell_column+=1
        else:
            continue
# Last iteration - Event
transfer_summary = dfBayAreaBeamEvents.get_ii_transfers()
for tour_purpose in transfer_summary.keys():
    if "work" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =11
            if "0" in transfr:
                cell_row="D"
            elif "1" in transfr:
                cell_row="E"
            elif "2" in transfr:
                cell_row="F"
            elif "3+" in transfr:
                cell_row="G"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "school" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =20
            if "0" in transfr:
                cell_row="D"
            elif "1" in transfr:
                cell_row="E"
            elif "2" in transfr:
                cell_row="F"
            elif "3+" in transfr:
                cell_row="G"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "univ" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =29
            if "0" in transfr:
                cell_row="D"
            elif "1" in transfr:
                cell_row="E"
            elif "2" in transfr:
                cell_row="F"
            elif "3+" in transfr:
                cell_row="G"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "shopping" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =38
            if "0" in transfr:
                cell_row="D"
            elif "1" in transfr:
                cell_row="E"
            elif "2" in transfr:
                cell_row="F"
            elif "3+" in transfr:
                cell_row="G"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "othmaint" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =47
            if "0" in transfr:
                cell_row="D"
            elif "1" in transfr:
                cell_row="E"
            elif "2" in transfr:
                cell_row="F"
            elif "3+" in transfr:
                cell_row="G"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "eatout" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =56
            if "0" in transfr:
                cell_row="D"
            elif "1" in transfr:
                cell_row="E"
            elif "2" in transfr:
                cell_row="F"
            elif "3+" in transfr:
                cell_row="G"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "social" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =65
            if "0" in transfr:
                cell_row="D"
            elif "1" in transfr:
                cell_row="E"
            elif "2" in transfr:
                cell_row="F"
            elif "3+" in transfr:
                cell_row="G"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "othdiscr" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =74
            if "0" in transfr:
                cell_row="D"
            elif "1" in transfr:
                cell_row="E"
            elif "2" in transfr:
                cell_row="F"
            elif "3+" in transfr:
                cell_row="G"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "escort" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =83
            if "0" in transfr:
                cell_row="D"
            elif "1" in transfr:
                cell_row="E"
            elif "2" in transfr:
                cell_row="F"
            elif "3+" in transfr:
                cell_row="G"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)

# First iteration - Event
transfer_summary = dfBayAreaZerothEvents.get_ii_transfers()
for tour_purpose in transfer_summary.keys():
    if "work" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =11
            if "0" in transfr:
                cell_row="N"
            elif "1" in transfr:
                cell_row="O"
            elif "2" in transfr:
                cell_row="P"
            elif "3+" in transfr:
                cell_row="Q"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "school" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =20
            if "0" in transfr:
                cell_row="N"
            elif "1" in transfr:
                cell_row="O"
            elif "2" in transfr:
                cell_row="P"
            elif "3+" in transfr:
                cell_row="Q"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "univ" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =29
            if "0" in transfr:
                cell_row="N"
            elif "1" in transfr:
                cell_row="O"
            elif "2" in transfr:
                cell_row="P"
            elif "3+" in transfr:
                cell_row="Q"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "shopping" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =38
            if "0" in transfr:
                cell_row="N"
            elif "1" in transfr:
                cell_row="O"
            elif "2" in transfr:
                cell_row="P"
            elif "3+" in transfr:
                cell_row="Q"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "othmaint" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =47
            if "0" in transfr:
                cell_row="N"
            elif "1" in transfr:
                cell_row="O"
            elif "2" in transfr:
                cell_row="P"
            elif "3+" in transfr:
                cell_row="Q"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "eatout" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =56
            if "0" in transfr:
                cell_row="N"
            elif "1" in transfr:
                cell_row="O"
            elif "2" in transfr:
                cell_row="P"
            elif "3+" in transfr:
                cell_row="Q"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "social" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =65
            if "0" in transfr:
                cell_row="N"
            elif "1" in transfr:
                cell_row="O"
            elif "2" in transfr:
                cell_row="P"
            elif "3+" in transfr:
                cell_row="Q"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "othdiscr" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =74
            if "0" in transfr:
                cell_row="N"
            elif "1" in transfr:
                cell_row="O"
            elif "2" in transfr:
                cell_row="P"
            elif "3+" in transfr:
                cell_row="Q"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)
    elif "escort" in tour_purpose:
        for transfr in transfer_summary.get(tour_purpose).keys():
            cell_column =83
            if "0" in transfr:
                cell_row="N"
            elif "1" in transfr:
                cell_row="O"
            elif "2" in transfr:
                cell_row="P"
            elif "3+" in transfr:
                cell_row="Q"
            nested_dict_pairs_populator(transfer_summary[tour_purpose][transfr],cell_row,cell_column)

In [189]:
# Populate noofTransfers summary
transfer_dict = dfBayAreaBeamEvents.get_ii_transfers_firstmile_summary()
transfer_wrksheet = workbook["noTransfer_Summary"]
for distance_cat,values in transfer_dict.iteritems():
    if isinstance(values,dict):
        if distance_cat=="<0.25mil":
            cell_row=11
            cell_col=4
        elif distance_cat==">=0.25mil & <0.5mil":
            cell_row=12
            cell_col=4
        elif distance_cat==">=0.5mil & <1mil":
            cell_row=13
            cell_col=4
        elif distance_cat==">=1mil":
            cell_row=14
            cell_col=4
        for k,v in transfer_dict.get(distance_cat).items():
            transfer_wrksheet.cell(row=(cell_row),column=(cell_col)).value=(v)*scale_factor
            cell_col+=1

transfer_dict = dfBayAreaZerothEvents.get_ii_transfers_firstmile_summary()
for distance_cat,values in transfer_dict.iteritems():
    if isinstance(values,dict):
        if distance_cat=="<0.25mil":
            cell_row=11
            cell_col=14
        elif distance_cat==">=0.25mil & <0.5mil":
            cell_row=12
            cell_col=14
        elif distance_cat==">=0.5mil & <1mil":
            cell_row=13
            cell_col=14
        elif distance_cat==">=1mil":
            cell_row=14
            cell_col=14
        for k,v in transfer_dict.get(distance_cat).items():
            transfer_wrksheet.cell(row=(cell_row),column=(cell_col)).value=(v)*scale_factor
            cell_col+=1

In [190]:
# save the data
workbook.save(BASE_DIR.parent.joinpath("exported", "summary_report_v10.0_trial.xlsx"))

In [17]:
# save the data
# workbook.save(BASE_DIR.parent.joinpath("exported", "summary_report_v10.0_trial.xlsx"))

In [ ]:
# def nested_dict_pairs_populator(dict_obj,row_no,col_no):
#     # Mode_Choice_wrksheet = sorted(dict_obj.items(),key=lambda kv:kv[1])
#     transfer_summary = OrderedDict(sorted(dict_obj.items())).copy()
#     cell_row=row_no
#     cell_column=col_no
#     for mode, count in transfer_summary.items():
#         if "<0.25mil" in mode:
#             transfer_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
#         elif ">=0.25mil & <0.5mil" in mode:
#             cell_column+=1
#             transfer_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
#         elif ">=0.5mil & <1mil" in mode:
#             cell_column+=1
#             transfer_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
#         elif ">=1mil" in mode:
#             cell_column+=1
#             transfer_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
#         else:
#             continue
# Populate noofTransfers by tourpurpose
# transfer_wrksheet = workbook["noTransfer_Summary"]
# #Last iteration - Event
# transfer_summary = dfBayAreaBeamEvents.get_ii_transfers_firstmile_summary()
# cell_column=11
# cell_row="D"
# nested_dict_pairs_populator(transfer_summary,cell_row,cell_column)

In [205]:
df = dfBayAreaBeamEvents.dfEvents
df = df[~df["person"].str.contains("ix",na=False) & ~df["person"].str.contains("rideHailAgent",na=False) & ~df["person"].str.contains("TransitDriverAgent",na=False) & (~df["person"].isna())].copy()
df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:]
# df = df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:]
# df["walkTransitDistances"]= df["tracingData"].str.rsplit(":",3)
# df["trip_cost_utility"]= df["tracingData"].str.rsplit(":",2)[1]
# df["numberofTransfers"]= df["tracingData"].str[-22:].str.split(")",expand=True)[0]

# df["walkTransitDistances"]= df["tracingData"].str.rsplit(":",expand=True)[3]
# df["trip_cost_utility"]= df["tracingData"].str.rsplit(":",expand=True)[1]
# df["numberofTransfers"]= df["tracingData"].str.rsplit(":",expand=True)[0]

# df["walkTransitDistances"]= df["tracingData"].str.split(":",expand=True)[0].str.split(",",expand=True)[1].str.split(")",expand=True)[0]
# df["trip_cost_utility"]= df["tracingData"].str.split(":",expand=True)[1].str.split(",",expand=True)[1].str.split(")",expand=True)[0]
# df["numberofTransfers"]= df["tracingData"].str.split(":",expand=True)[2].str.split(",",expand=True)[1].str.split(")",expand=True)[0]
#
# df["walkTransitDistances"] = pd.to_numeric(df["walkTransitDistances"], errors='coerce').replace(np.nan,0,regex=True)
# df["trip_cost_utility"] = pd.to_numeric(df["trip_cost_utility"], errors='coerce').replace(np.nan,0,regex=True)
# df["numberofTransfers"] = pd.to_numeric(df["numberofTransfers"], errors='coerce').replace(np.nan,0,regex=True).round(0).astype(int)

# dData = df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:][["tracingData","walkTransitDistances","trip_cost_utility","numberofTransfers"]]
# df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:][["tracingData","walkTransitDistances","trip_cost_utility","numberofTransfers"]]

# df["numberofTransfers"]=df["tracingData"].str[-22:].str.split(")",expand=True)[0]
# df["travel_cost"]= df["tracingData"].str[-34:-25]

,person,vehicle,time,type,link,facility,actType,driver,parkingType,chargingPointType,...,departureTime,fromStopIndex,secondaryFuel,primaryFuelType,secondaryFuelType,numPassengers,links,primaryFuel,riders,Travel_Time
37666,111327-1,NaN,12420.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39788,371102-1,NaN,15720.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40151,203118-2,NaN,16140.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40689,9486-2,NaN,16680.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41282,153922-1,NaN,17220.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864486,284190-1,NaN,85544.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
864846,2749151-1,NaN,85620.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
866456,43434-1,NaN,85980.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
866716,259013-2,NaN,86040.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
df["noofTransfers"]=df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:]["tracingData"].str[-22:].str.split(")",expand=True)[0].str.split(",",expand=True)[1]
df["travelCosts"]=df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:]["tracingData"].str[-34:-25].str.split(",",expand=True)[1]
df["walkTransitDistance"] = df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:]["tracingData"].str[1:-37].str.split(",",expand=True)[1].str.split(":",expand=True)[0]
df.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:][["tracingData","walkTransitDistance","travelCosts","noofTransfers"]]

,tracingData,walkTransitDistance,travelCosts,noofTransfers
37666,"(walkTransitDistances,1534.533:513.774:0.0):(c...",1534.533,2.75,1.0
39788,"(walkTransitDistances,837.588:75.841):(cost,2....",837.588,2.75,0.0
40151,"(walkTransitDistances,977.444:254.281):(cost,2...",977.444,2.75,0.0
40689,"(walkTransitDistances,302.271:282.058):(cost,2...",302.271,2.75,0.0
41282,"(walkTransitDistances,283.006:31.274:280.166):...",283.006,2.75,1.0
...,...,...,...,...
864486,"(walkTransitDistances,120.476:294.186):(cost,2...",120.476,2.75,0.0
864846,"(walkTransitDistances,149.215:0.0):(cost,2.75)...",149.215,2.75,0.0
866456,"(walkTransitDistances,126.91:199.603):(cost,2....",126.91,2.75,0.0
866716,"(walkTransitDistances,376.635:71.104:794.922):...",376.635,2.75,1.0


In [7]:
dfLast = dfBayAreaBeamEvents.dfEvents
dfFirst = dfBayAreaZerothEvents.dfEvents

In [31]:
dfLast.loc[(~dfLast["person"].str.contains("ix",na=False)) & (~dfLast["person"].str.contains("rideHailAgent",na=False))
           & (~dfLast["person"].str.contains("TransitDriverAgent",na=False)) & (~dfLast["person"].isna())
           & (dfLast["type"]=="TripArrivalEvent"),"mode"].value_counts().sum()*20

666400

In [32]:
dfFirst.loc[(~dfFirst["person"].str.contains("ix",na=False)) & (~dfFirst["person"].str.contains("rideHailAgent",na=False))
            & (~dfFirst["person"].str.contains("TransitDriverAgent",na=False)) & (~dfFirst["person"].isna())
            & (dfFirst["type"]=="TripArrivalEvent"),"mode"].value_counts().sum()*20

665800

In [35]:
df2 = dfLast.loc[(~dfLast["person"].str.contains("ix",na=False)) & (~dfLast["person"].str.contains("rideHailAgent",na=False))
           & (~dfLast["person"].str.contains("TransitDriverAgent",na=False)) & (~dfLast["person"].isna())
           & (dfLast["type"]=="TripArrivalEvent"),]

df1 = dfFirst.loc[(~dfFirst["person"].str.contains("ix",na=False)) & (~dfFirst["person"].str.contains("rideHailAgent",na=False))
            & (~dfFirst["person"].str.contains("TransitDriverAgent",na=False)) & (~dfFirst["person"].isna())
            & (dfFirst["type"]=="TripArrivalEvent"),]

df3 = pd.concat([df1,df2]).drop_duplicates(keep=False)

df4 = pd.concat([df2,df1]).drop_duplicates(keep=False)

In [46]:
df2.to_csv(BASE_DIR.parent.joinpath("exported", "Last_iteration.10_trial.csv"))
df1.to_csv(BASE_DIR.parent.joinpath("exported", "First_iteration.10_trial.csv"))

In [ ]:
# tour_purpose_dict = dfTour.groupby(["tourPurpose", "vehicleOwnership", "mode"]).size().reset_index().rename(columns={0: "count"})\
#                           .groupby("tourPurpose")\
#                           .apply(lambda vehOwntype: dict(vehOwntype.groupby("vehicleOwnership")
#                                                          .apply(lambda mode_used: dict(zip(mode_used["mode"], mode_used["count"]))))).copy()

In [ ]:
# dfEvents = self.dfEvents.copy()
# dfTour = dfEvents.loc[(~dfEvents["person"].str.contains("ix",na=False) &
#                        ~dfEvents["person"].str.contains("rideHailAgent",na=False) &
#                        ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
#                        (~dfEvents["person"].isna())),:]
#     .dropna(subset = ["vehicleOwnership"])
# dfTour=dfTour[dfTour["type"]=="TripArrivalEvent"]
# tour_purpose_dict = dfTour.groupby(["tourPurpose", "vehicleOwnership", "mode"]).size().reset_index().rename(columns={0: "count"})
#     .groupby("tourPurpose")
#     .apply(lambda vehOwntype: dict(vehOwntype.groupby("vehicleOwnership")
#                                    .apply(lambda mode_used: dict(zip(mode_used["mode"], mode_used["count"]))))).copy()
# for key,value in tour_purpose_dict.iteritems():
#     if isinstance(tour_purpose_dict.get(key),dict):
#         for auto_ownership,modes in tour_purpose_dict.get(key).items():
#             mode_keys = ["car", "walk_transit", "drive_transit", "bike_transit", "ride_hail_transit", "walk", "bike", "ride_hail_pooled", "ride_hail", "hov2_teleportation", "hov2","hov3_teleportation", "hov3"]
#             if isinstance(modes,dict):
#                 for mode,count in modes.items():
#                     if mode in mode_keys:
#                         mode_keys.remove(mode)
#                 for mode in mode_keys:
#                     tour_purpose_dict[key][auto_ownership][mode]=0

In [60]:
(dfLast.loc[(~dfLast["person"].str.contains("ix",na=False)) & (~dfLast["person"].str.contains("rideHailAgent",na=False))
            & (~dfLast["person"].str.contains("TransitDriverAgent",na=False)) & (~dfLast["person"].isna())
            & (dfLast["type"]=="TripArrivalEvent"),].groupby(["tourPurpose", "vehicleOwnership", "mode"]).size())

tourPurpose  vehicleOwnership  mode              
eatout       auto_deficient    bike                   15
                               bike_transit           20
                               car                   454
                               hov2                   29
                               hov2_teleportation      6
                                                    ... 
work         no_auto           hov2_teleportation     78
                               hov3_teleportation    159
                               ride_hail              21
                               walk                  919
                               walk_transit          264
Length: 200, dtype: int64

In [54]:
# df2.groupby(["tourPurpose", "vehicleOwnership", "mode"]).size().reset_index().rename(columns={0: "count"})
(dfLast.loc[(~dfLast["person"].str.contains("ix",na=False)) & (~dfLast["person"].str.contains("rideHailAgent",na=False))
           & (~dfLast["person"].str.contains("TransitDriverAgent",na=False)) & (~dfLast["person"].isna())
           & (dfLast["type"]=="TripArrivalEvent"),].groupby(["tourPurpose", "vehicleOwnership", "mode"]).size()*20).to_csv(BASE_DIR.parent.joinpath("exported", "Last_iteration_tour.10_trial.csv"))
# df2.groupby(["tourPurpose", "vehicleOwnership", "mode"]).size()*20

In [55]:
(dfFirst.loc[(~dfFirst["person"].str.contains("ix",na=False)) & (~dfFirst["person"].str.contains("rideHailAgent",na=False))
            & (~dfFirst["person"].str.contains("TransitDriverAgent",na=False)) & (~dfFirst["person"].isna())
            & (dfFirst["type"]=="TripArrivalEvent"),].groupby(["tourPurpose", "vehicleOwnership", "mode"]).size()*20).to_csv(BASE_DIR.parent.joinpath("exported", "First_iteration_tour.10_trial.csv"))

In [57]:
pd.DataFrame(dfBayAreaZerothEvents.get_ii_tourpurpose_stats()).to_csv(BASE_DIR.parent.joinpath("exported", "First_iteration_tour_aftr.10_trial.csv"))
pd.DataFrame(dfBayAreaBeamEvents.get_ii_tourpurpose_stats()).to_csv(BASE_DIR.parent.joinpath("exported", "Last_iteration_tour_aftr.10_trial.csv"))

In [50]:
df1.groupby(["tourPurpose", "vehicleOwnership", "mode"]).size().reset_index().rename(columns={0: "count"})["count"].sum()

33290

In [39]:
df3

,person,vehicle,time,type,link,facility,actType,driver,parkingType,chargingPointType,...,departureTime,fromStopIndex,secondaryFuel,primaryFuelType,secondaryFuelType,numPassengers,links,primaryFuel,riders,Travel_Time
30882,406508-1,NaN,664.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30901,354568-1,NaN,692.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30941,1892734-3,NaN,971.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30948,386899-1,NaN,1019.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30976,107063-1,NaN,1073.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891258,568428-3,NaN,106115.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
891351,2566347-1,NaN,106351.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
891418,568428-3,NaN,106530.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
891578,2566347-1,NaN,107079.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df4

,person,vehicle,time,type,link,facility,actType,driver,parkingType,chargingPointType,...,departureTime,fromStopIndex,secondaryFuel,primaryFuelType,secondaryFuelType,numPassengers,links,primaryFuel,riders,Travel_Time
30828,240750-5,NaN,155.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30844,236959-1,NaN,209.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30870,320377-1,NaN,265.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30898,162419-2,NaN,290.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30969,366469-1,NaN,630.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881818,2566347-1,NaN,107073.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
881879,2472490-1,NaN,107211.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
881979,1404552-1,NaN,107546.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
882023,2472490-1,NaN,107672.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
dfLast.loc[dfLast["person"]=="100050-3"]

,person,vehicle,time,type,link,facility,actType,driver,parkingType,chargingPointType,...,departureTime,fromStopIndex,secondaryFuel,primaryFuelType,secondaryFuelType,numPassengers,links,primaryFuel,riders,Travel_Time
86698,100050-3,NaN,25320.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86699,100050-3,NaN,25320.0,actend,21696.0,NaN,home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89168,100050-3,NaN,25511.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89169,100050-3,NaN,25511.0,actstart,21270.0,NaN,school,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465930,100050-3,NaN,51840.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465939,100050-3,NaN,51840.0,actend,21270.0,NaN,school,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478238,100050-3,NaN,52766.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478239,100050-3,NaN,52766.0,actstart,21696.0,NaN,home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658478,100050-3,NaN,64020.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658479,100050-3,NaN,64020.0,actend,21696.0,NaN,home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
dfFirst.loc[dfFirst["person"]=="100050-3"]

,person,vehicle,time,type,link,facility,actType,driver,parkingType,chargingPointType,...,departureTime,fromStopIndex,secondaryFuel,primaryFuelType,secondaryFuelType,numPassengers,links,primaryFuel,riders,Travel_Time
84206,100050-3,NaN,25320.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84214,100050-3,NaN,25320.0,actend,21697.0,NaN,home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84216,100050-3,body-100050-3,25320.0,PersonEntersVehicle,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97132,100050-3,body-100050-3,26253.0,PersonLeavesVehicle,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97141,100050-3,NaN,26253.0,TripArrivalEvent,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97142,100050-3,NaN,26253.0,actstart,21271.0,NaN,school,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
453695,100050-3,NaN,51840.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
453725,100050-3,NaN,51840.0,actend,21270.0,NaN,school,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
453788,100050-3,body-100050-3,51840.0,PersonEntersVehicle,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465856,100050-3,body-100050-3,52766.0,PersonLeavesVehicle,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
